## 1. Decision Tree for Classification

In [6]:
import pandas as pd
import numpy as np

# Dữ liệu từ bảng
data = {
    'Age': [23, 25, 27, 29, 29],
    'Likes English': [0, 1, 1, 0, 0],
    'Likes AI': [0, 1, 0, 1, 0],
    'Raise Salary': [0, 0, 1, 1, 0]
}

# Tạo DataFrame
df = pd.DataFrame(data)

# Hiển thị DataFrame
df


,Age,Likes English,Likes AI,Raise Salary
0,23,0,0,0
1,25,1,1,0
2,27,1,0,1
3,29,0,1,1
4,29,0,0,0


In [2]:
# Tính chỉ số Gini cho cột 'Raise Salary'
def gini(column):
    # Đếm tần suất xuất hiện của mỗi giá trị trong cột
    counts = column.value_counts(normalize=True)
    # Áp dụng công thức Gini
    gini_value = 1 - sum(counts**2)
    return gini_value

# Tính chỉ số Gini cho cột 'Raise Salary'
gini_salary = gini(df['Raise Salary'])
print(f"Chỉ số Gini cho cột 'Raise Salary' là: {gini_salary:.2f}")

Chỉ số Gini cho cột 'Raise Salary' là: 0.48


In [3]:
# Tính chỉ số Gini cho thuộc tính 'Likes English' làm node gốc
def gini_for_split(df, split_column, target_column):
    # Nhóm dữ liệu dựa trên giá trị của cột phân tách
    groups = df.groupby(split_column)[target_column]
    
    # Tính chỉ số Gini cho từng nhóm
    weighted_gini = 0
    total_samples = len(df)
    
    for group, data in groups:
        group_size = len(data)
        group_gini = gini(data)
        weighted_gini += (group_size / total_samples) * group_gini
    
    return weighted_gini

# Tính Gini cho bộ dữ liệu khi 'Likes English' là node gốc
gini_likes_english = gini_for_split(df, 'Likes English', 'Raise Salary')
print(f"Chỉ số Gini cho node gốc 'Likes English' là: {gini_likes_english:.2f}")

Chỉ số Gini cho node gốc 'Likes English' là: 0.47


In [4]:
def gini_for_age_split(df, age_column, target_column, threshold):
    # Chia tập dữ liệu thành D1 và D2
    D1 = df[df[age_column] <= threshold][target_column]
    D2 = df[df[age_column] > threshold][target_column]
    
    # Tính Gini cho D1 và D2
    gini_D1 = gini(D1)
    gini_D2 = gini(D2)
    
    # Tính Gini có trọng số dựa trên số lượng mẫu
    total_samples = len(df)
    weighted_gini = (len(D1) / total_samples) * gini_D1 + (len(D2) / total_samples) * gini_D2
    
    return weighted_gini

# Tính Gini cho bộ dữ liệu khi 'Age' là node gốc và phân chia theo 'Age <= 26'
gini_age_split = gini_for_age_split(df, 'Age', 'Raise Salary', 26)
print(f"Chỉ số Gini cho node gốc 'Age' với điều kiện 'Age <= 26' là: {gini_age_split:.2f}")

Chỉ số Gini cho node gốc 'Age' với điều kiện 'Age <= 26' là: 0.27


In [7]:

# Hàm tính Entropy
def entropy(column):
    # Tính tần suất (xác suất) của từng lớp
    counts = column.value_counts(normalize=True)
    # Tính Entropy theo công thức
    entropy_value = -sum(counts * np.log2(counts))
    return entropy_value

# Tính Entropy cho cột 'Raise Salary'
entropy_value = entropy(df['Raise Salary'])
print(f"Giá trị Entropy của cột 'Raise Salary' là: {entropy_value:.2f}")

Giá trị Entropy của cột 'Raise Salary' là: 0.97


In [8]:
entropy_original = entropy(df['Raise Salary'])

# Phân chia dữ liệu theo thuộc tính 'Likes English'
entropy_values = {}
for value in df['Likes English'].unique():
    subset = df[df['Likes English'] == value]
    entropy_values[value] = entropy(subset['Raise Salary'])

# Tính Gain
weighted_entropy_sum = sum((len(df[df['Likes English'] == value]) / len(df)) * entropy
                            for value, entropy in entropy_values.items())
gain = entropy_original - weighted_entropy_sum

print(f'Gain của thuộc tính "Likes English": {gain}')

Gain của thuộc tính "Likes English": 0.01997309402197489


In [9]:
from sklearn import datasets
iris_X, iris_y = datasets.load_iris(return_X_y=True)


In [10]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

# Load the iris dataset
iris_X, iris_y = datasets.load_iris(return_X_y=True)

# Split train : test = 8:2
X_train, X_test, y_train, y_test = train_test_split(
    iris_X, iris_y,
    test_size=0.2,
    random_state=42
)

# Define model
dt_classifier = DecisionTreeClassifier()

# Train
dt_classifier.fit(X_train, y_train)

# Predict and evaluate
y_pred = dt_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Accuracy: 1.0


## 2. Decision Tree for Regression

In [11]:
import pandas as pd

# Tạo dữ liệu
data = {
    'Age': [23, 25, 27, 29, 29],
    'Likes English': [0, 1, 1, 0, 0],
    'Likes AI': [0, 1, 0, 1, 0],
    'Salary': [200, 400, 300, 500, 400]
}

# Tạo DataFrame
df = pd.DataFrame(data)

# Hiển thị DataFrame
df


,Age,Likes English,Likes AI,Salary
0,23,0,0,200
1,25,1,1,400
2,27,1,0,300
3,29,0,1,500
4,29,0,0,400


In [12]:
# Tính trung bình Salary cho từng nhóm 'Likes AI'
mean_salary_likes_ai_1 = df[df['Likes AI'] == 1]['Salary'].mean()
mean_salary_likes_ai_0 = df[df['Likes AI'] == 0]['Salary'].mean()

# Tính SSE cho từng nhóm
sse_likes_ai_1 = ((df[df['Likes AI'] == 1]['Salary'] - mean_salary_likes_ai_1) ** 2).sum()
sse_likes_ai_0 = ((df[df['Likes AI'] == 0]['Salary'] - mean_salary_likes_ai_0) ** 2).sum()

# Tổng SSE
total_sse = sse_likes_ai_1 + sse_likes_ai_0

print(f"Total SSE: {total_sse}")

Total SSE: 25000.0


In [14]:
# Chia dữ liệu thành hai nhóm
D1 = df[df['Age'] <= 24]
D2 = df[df['Age'] > 24]

# Tính trung bình Salary cho từng nhóm
mean_salary_D1 = D1['Salary'].mean() if not D1.empty else 0
mean_salary_D2 = D2['Salary'].mean() if not D2.empty else 0

# Tính SSE cho từng nhóm
sse_D1 = ((D1['Salary'] - mean_salary_D1) ** 2).sum()
sse_D2 = ((D2['Salary'] - mean_salary_D2) ** 2).sum()

# Tổng SSE
total_sse = sse_D1 + sse_D2

print(f"Total SSE: {total_sse}")

Total SSE: 20000.0
